In [4]:
import requests
import airtable
import datetime

# Airtable setup
AIRTABLE_BASE_ID = 'app7mcJMf9O8GQRal'
AIRTABLE_API_KEY = ''
TABLE_NAME = 'tiktok_user'
VIEW_NAME = '0_no_info'

# Tiktok key
x-rapidapi-key = ""

# TikTok API setup
TIKTOK_API_URL = "https://tiktok-api23.p.rapidapi.com/api/user/info"
HEADERS = {
    "x-rapidapi-key": x-rapidapi-key,
    "x-rapidapi-host": "tiktok-api23.p.rapidapi.com"
}

def get_airtable_data():
    at = airtable.Airtable(AIRTABLE_BASE_ID, TABLE_NAME, api_key=AIRTABLE_API_KEY)
    records = at.get_all(view=VIEW_NAME)
    return records

def update_airtable_record(record_id, data):
    at = airtable.Airtable(AIRTABLE_BASE_ID, TABLE_NAME, api_key=AIRTABLE_API_KEY)
    at.update(record_id, data)

def get_tiktok_user_info(username):
    querystring = {"uniqueId": username}
    response = requests.get(TIKTOK_API_URL, headers=HEADERS, params=querystring)
    return response.json()

def transform_timestamp_to_datetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

def main():
    airtable_records = get_airtable_data()
    
    for record in airtable_records:
        username = record['fields'].get('username')
        if username:
            # Get TikTok user info
            user_info_response = get_tiktok_user_info(username)
            user_info = user_info_response.get('userInfo', {}).get('user', {})
            stats_info = user_info_response.get('userInfo', {}).get('stats', {})
            
            if user_info:
                # Prepare data to be updated
                data = {
                    'id': user_info.get('id', ''),
                    'nickname': user_info.get('nickname', ''),
                    'secUid': user_info.get('secUid', ''),
                    'avatarLarger': user_info.get('avatarLarger', ''),
                    'createTime': transform_timestamp_to_datetime(user_info.get('createTime', 0)),
                    'verified': str(user_info.get('verified', False)),
                    'bioLink_link': user_info.get('bioLink', {}).get('link', ''),
                    'followerCount': stats_info.get('followerCount', 0),
                    'followingCount': stats_info.get('followingCount', 0),
                    'heart': stats_info.get('heart', 0),
                    'heartCount': stats_info.get('heartCount', 0),
                    'videoCount': stats_info.get('videoCount', 0),
                    'diggCount': stats_info.get('diggCount', 0)
                }
                
                # Update Airtable record
                update_airtable_record(record['id'], data)
                print(f"Updated record for username: {username}")

if __name__ == "__main__":
    main()


Updated record for username: quynhanhthipham25.11
